# Keras Intensive Run

Instructions
1. Define the global variables;
2. Run the second snippet. It builds the network and saves the output to a folder displayed on the console.

In [23]:
# GLOBAL VARIABLES
labelsGroup = [
    ["angry", "fear", "happy", "neutral", "sad", "surprise"],
    ["happy", "sad"],
]
iterationsGroup = [1000, 2500, 5000]
hiddenLayersGroup = [
    [144],
    [300,200,100,50],
]

In [24]:
def confusionMatrix(labels, pred, y):
    
    confusionTFPN = {}
    confusionLabels = {}
    
    # Foreach emotion
    for ie in range(len(labels)):
        # TRUE AND FALSE POSITIVES AND NEGATIVES (TFPN)
        
        # Get indexes where emotion was predicted
        ieIndexesPred = [i for i in range(pred.size) if pred[i]==ie]
        ieIndexesNotPred = [i for i in range(pred.size) if pred[i]!=ie]

        # Predicted, and Actual
        TP = sum(pred[:,np.newaxis][ieIndexesPred]==y[ieIndexesPred])
        # Predicted, but not actual
        FP = sum(pred[:,np.newaxis][ieIndexesPred]!=y[ieIndexesPred])
        # Not predicted, but actual
        FN = sum(y[ieIndexesNotPred]==ie)
        # Not predicted and not actual
        TN = sum(y[ieIndexesNotPred]!=ie)

        TP = TP[0] if TP else 0
        FP = FP[0] if FP else 0
        FN = FN[0] if FN else 0
        TN = TN[0] if TN else 0
        
        confusionTFPN[labels[ie]] = {
            'TP': TP,
            'FP': FP,
            'FN': FN,
            'TN': TN
        }
        
        # CONFUSION WITH OTHER labels (Confusion)
        # For emotion e
        # Count the number of predictions made for all classes
        # Foreach emotion, check how many times it has been predicted 
        
        # Get indexes where emotion is real
        ieIndexesY = [i for i in range(y.size) if y[i]==ie]
        
        # Foreach matching prediction, check what emotion was predicted
        confusionLabels[labels[ie]] = {
            e: sum(pred[:,np.newaxis][ieIndexesY]==labels.index(e))[0] for e in labels
        }
        
    return confusionTFPN, confusionLabels

def outputConfusionTables(labels, testExamplesNumber, confusionTFPN, confusionLabels, axs, axsLine, axsTitle):
     # TABLES
    rows = tuple(labels)
    
    # TABLES / TFPN
    # Output confusion matrix as plot table
    data = [
        [
            f"{vals['TP']} ({vals['TP']/testExamplesNumber*100:.2f}%)",
            f"{vals['TN']} ({vals['TN']/testExamplesNumber*100:.2f}%)",
            f"{vals['TP']+vals['TN']} ({(vals['TP']+vals['TN'])/testExamplesNumber*100:.2f}%)",
            f"{vals['FP']} ({vals['FP']/testExamplesNumber*100:.2f}%)",
            f"{vals['FN']} ({vals['FN']/testExamplesNumber*100:.2f}%)",
            f"{vals['FP']+vals['FN']} ({(vals['FP']+vals['FN'])/testExamplesNumber*100:.2f}%)",
            vals['FP']+vals['FN']+vals['TP']+vals['TN']
        ]
        for _, vals in confusionTFPN.items()
    ]
    columns = ['TP (%)', 'TN (%)', 'T (%)', 'FP (%)', 'FN (%)', 'F (%)', f"Total"]
    
    # Colorize cells depending on value
    cellColours=plt.cm.Blues([
        [
            vals['TP']/testExamplesNumber,
            vals['TN']/testExamplesNumber,
            0,
            vals['FP']/testExamplesNumber,
            vals['FN']/testExamplesNumber,
            0,
            0
        ]
        for _, vals in confusionTFPN.items()
    ])
        
    the_table = axs[axsLine].table(
      cellText=data,
      rowLabels=rows,
      colLabels=columns,
      loc='center',
      cellColours=cellColours
    )
    # the_table.scale(1.2, 1)
    the_table.auto_set_font_size(False)
    the_table.set_fontsize(20)
    axs[axsLine].axis('off')
    axs[axsLine].axis('tight')
    axs[axsLine].set_title(f"Confusion matrix {axsTitle}", fontsize=20, pad=0)    
    
    # TABLES / Confusion    
    # Output confusion matrix as plot table
    data = [[o for _,o in others.items()] for _,others in confusionLabels.items()]
    columns = rows
    
    # Colorize cells depending on value
    vals = [o for _,others in confusionLabels.items() for _,o in others.items()]
    normal = plt.Normalize(min(vals)-1, max(vals)+1)
    cellColours=plt.cm.Blues(normal(data))
        
    the_table = axs[axsLine+1].table(
      cellText=data,
      rowLabels=rows,
      colLabels=columns,
      loc='center',
      cellColours=cellColours
    )
    
    the_table.set_fontsize(20)
    # the_table.scale(1, 4)
    axs[axsLine+1].axis('off')
    axs[axsLine+1].axis('tight')
    axs[axsLine+1].set_title(f"True/Predicted {axsTitle}", fontsize=20)
    

In [ ]:
# IMPORT LIBRARIES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#to load matlab mat files
from scipy.io import loadmat
# for neural network
import tensorflow as tf
from tensorflow import keras
# for saving metrics
import os 
import json

if not os.path.exists('./metrics'):
        os.makedirs('./metrics')

for labelNames in labelsGroup:
    # LOAD FILES
    print("\n\nLABELS", ', '.join(labelNames))

    # Load Training data
    mat=loadmat(f"../datasets/train/{'_'.join(labelNames)}.mat")
    # mat is a dict with key "X" for x-values, and key "y" for y values
    X, y = mat["X"], mat["y"]

    # Load Dev Data
    matDev=loadmat(f"../datasets/dev/{'_'.join(labelNames)}.mat")
    X_valid, y_valid = matDev["X"], matDev["y"]

    # Load Test Data
    matTest=loadmat(f"../datasets/test/{'_'.join(labelNames)}.mat")
    X_test, y_test = matTest["X"], matTest["y"]
    
    # NUMBERS
    m = X.shape[0] # number of training examples
    labels = np.max(y)+1 # number of labels
    features = X.shape[1] # number of features per example

    print(f"Loaded {m} traing examples with {labels} labels, each with {features} features (pixels).")
    print(f"Loaded {X_valid.shape[0]} dev examples.")
    print(f"Loaded {X_test.shape[0]} test examples.")
    
    print()
    
    # Create folder for metrics
    folder = f"./metrics/{'_'.join(labelNames)}"
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    for iterations in iterationsGroup:
        for hiddenLayers in hiddenLayersGroup:
            
            hiddenLayersText = '_'.join(hiddenLayers) if len(hiddenLayers)>1 else hiddenLayers[0]
            
            # Create folder for metrics
            folder = f"{folder}/{iterations}i_{len(hiddenLayers)}hl__{hiddenLayersText}"
            if not os.path.exists(folder):
                os.makedirs(folder)
            
            print(f"\nBuilding network for {iterations} iterations and {len(hiddenLayers)} hidden layers: {hiddenLayersText}...\n")
            
            # BUILD NETWORK
            # Create model
            modelSeq = []
            # Flattens each image (48x48) to 1x2304
            modelSeq.append(keras.layers.Flatten(input_shape = [48, 48]))
            # Hidden layers with relu activation function
            for h in hiddenLayers:
                modelSeq.append(keras.layers.Dense(h, activation = "relu" ))
            # Output layer with softmax activation function
            modelSeq.append(keras.layers.Dense(labels, activation = "softmax" ))

            model = keras.models.Sequential(modelSeq)

            # Compile model
            model.compile(
                # Using sparse categorical crossentropy loss function
                loss = "sparse_categorical_crossentropy",
                # Using stochastic gradient descent as gradient descent
                optimizer = "sgd",
                # In addition to cost, we want accuracy to help understanding how the model is working 
                metrics = ["accuracy"]
            )

            # Train the network
            history = model.fit(
                X,
                y,
                epochs = iterations,
                # batch_size = iteratios/10,
                validation_data = (X_valid, y_valid)
            )

            # METRICS (SAVE TO FILE)
            
            # Model training history
            with open(f"{folder}/history.json", "w") as f:
                json.dump(history.history, f)
            
            # Test model evaluation
            with open(f"{folder}/evaluation.json", "w") as f:
                json.dump(model.evaluate(X_test, y_test, return_dict=True), f)
            
            # Prediction for test data
            with open(f"{folder}/predict.json", "w") as f:
                json.dump(model.predict(X_test).tolist(), f)
                
            print(f"\nModel trained and metrics saved to {folder}!")
            
            # GENERATE GRAPHS AND SAVE TO FILE
            
            # Model training history
            pd.DataFrame(history.history).plot(figsize = (16, 10))
            plt.grid(True)
            plt.gca().set_ylim(0, 1)
            plt.title("Neural network training metrics")
            plt.savefig(f"{folder}/history.png")
            
            # Prediction for test data
            predict = model.predict(X_test)
            y_pred = np.array([np.argmax(p) for p in predict])
            confusionTFPN, confusionLabels = confusionMatrix(labelNames, y_pred, y_test)
            fig, axs = plt.subplots(2,1)
            outputConfusionTables(labelNames, y_test.size, confusionTFPN, confusionLabels, axs, 0, "WITHOUT regularization")
            fig.set_size_inches(18,6*len(labelNames))
            fig.subplots_adjust(left=0.2, top=20)
            fig.tight_layout()
            fig.savefig(f"{folder}/predict.png")



LABELS happy, sad
Loaded 2000 traing examples with 2 labels, each with 2304 features (pixels).
Loaded 400 dev examples.
Loaded 400 test examples.


Building network for 1000 iterations and 1 hidden layers: 144...

Epoch 1/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.8439 - accuracy: 0.4943 - val_loss: 0.8233 - val_accuracy: 0.5050
Epoch 2/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.6900 - accuracy: 0.5674 - val_loss: 0.6927 - val_accuracy: 0.5300
Epoch 3/1000
63/63 [==============================] - 0s 2ms/step - loss: 0.6721 - accuracy: 0.6006 - val_loss: 0.6199 - val_accuracy: 0.6775
Epoch 4/1000
63/63 [==============================] - 0s 2ms/step - loss: 0.6456 - accuracy: 0.6382 - val_loss: 0.6479 - val_accuracy: 0.5850
Epoch 5/1000
63/63 [==============================] - 0s 2ms/step - loss: 0.6306 - accuracy: 0.6457 - val_loss: 0.6364 - val_accuracy: 0.6225
Epoch 6/1000
63/63 [==============================] - 0s 2ms/step - loss: 

63/63 [==============================] - 0s 2ms/step - loss: 0.4723 - accuracy: 0.7766 - val_loss: 0.5480 - val_accuracy: 0.6975
Epoch 50/1000
63/63 [==============================] - 0s 2ms/step - loss: 0.4616 - accuracy: 0.7898 - val_loss: 0.5385 - val_accuracy: 0.7125
Epoch 51/1000
63/63 [==============================] - 0s 2ms/step - loss: 0.4272 - accuracy: 0.8205 - val_loss: 0.5530 - val_accuracy: 0.7475
Epoch 52/1000
63/63 [==============================] - 0s 6ms/step - loss: 0.4198 - accuracy: 0.8143 - val_loss: 0.5465 - val_accuracy: 0.7300
Epoch 53/1000
63/63 [==============================] - 0s 2ms/step - loss: 0.4123 - accuracy: 0.8241 - val_loss: 0.5881 - val_accuracy: 0.6650
Epoch 54/1000
63/63 [==============================] - 0s 2ms/step - loss: 0.4265 - accuracy: 0.8051 - val_loss: 0.6451 - val_accuracy: 0.6775
Epoch 55/1000
63/63 [==============================] - 0s 2ms/step - loss: 0.4405 - accuracy: 0.7893 - val_loss: 0.5392 - val_accuracy: 0.7075
Epoch 56/1000

63/63 [==============================] - 0s 5ms/step - loss: 0.2414 - accuracy: 0.9078 - val_loss: 1.2283 - val_accuracy: 0.5725
Epoch 107/1000
63/63 [==============================] - 0s 5ms/step - loss: 0.3193 - accuracy: 0.8863 - val_loss: 0.6096 - val_accuracy: 0.7325
Epoch 108/1000
63/63 [==============================] - 0s 5ms/step - loss: 0.2333 - accuracy: 0.9119 - val_loss: 0.5857 - val_accuracy: 0.7075
Epoch 109/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.2177 - accuracy: 0.9322 - val_loss: 0.7417 - val_accuracy: 0.7125
Epoch 110/1000
63/63 [==============================] - 0s 5ms/step - loss: 0.2566 - accuracy: 0.8996 - val_loss: 0.6076 - val_accuracy: 0.7150
Epoch 111/1000
63/63 [==============================] - 0s 6ms/step - loss: 0.2258 - accuracy: 0.9291 - val_loss: 0.5861 - val_accuracy: 0.7300
Epoch 112/1000
63/63 [==============================] - 0s 5ms/step - loss: 0.2297 - accuracy: 0.9149 - val_loss: 0.6025 - val_accuracy: 0.7125
Epoch 1

Epoch 163/1000
63/63 [==============================] - 0s 5ms/step - loss: 0.1312 - accuracy: 0.9628 - val_loss: 0.7263 - val_accuracy: 0.7125
Epoch 164/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.0956 - accuracy: 0.9849 - val_loss: 0.7559 - val_accuracy: 0.7350
Epoch 165/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.1001 - accuracy: 0.9751 - val_loss: 0.7131 - val_accuracy: 0.7000
Epoch 166/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.1077 - accuracy: 0.9843 - val_loss: 0.7246 - val_accuracy: 0.7400
Epoch 167/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0877 - accuracy: 0.9839 - val_loss: 0.7199 - val_accuracy: 0.7175
Epoch 168/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.1043 - accuracy: 0.9786 - val_loss: 0.7165 - val_accuracy: 0.7350
Epoch 169/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.1195 - accuracy: 0.9686 - val_loss: 0.8094 - val_accuracy:

Epoch 220/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0445 - accuracy: 0.9980 - val_loss: 0.8111 - val_accuracy: 0.7300
Epoch 221/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0449 - accuracy: 0.9965 - val_loss: 0.8341 - val_accuracy: 0.7200
Epoch 222/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0440 - accuracy: 0.9957 - val_loss: 0.8890 - val_accuracy: 0.7050
Epoch 223/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0646 - accuracy: 0.9873 - val_loss: 0.8244 - val_accuracy: 0.7300
Epoch 224/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0632 - accuracy: 0.9874 - val_loss: 0.7200 - val_accuracy: 0.7350
Epoch 225/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0659 - accuracy: 0.9921 - val_loss: 0.8190 - val_accuracy: 0.7275
Epoch 226/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0449 - accuracy: 0.9926 - val_loss: 0.8352 - val_accuracy:

Epoch 277/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.0245 - accuracy: 0.9989 - val_loss: 0.9093 - val_accuracy: 0.7300
Epoch 278/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0234 - accuracy: 1.0000 - val_loss: 0.9253 - val_accuracy: 0.7200
Epoch 279/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0207 - accuracy: 0.9999 - val_loss: 0.9120 - val_accuracy: 0.7150
Epoch 280/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.0237 - accuracy: 1.0000 - val_loss: 0.9162 - val_accuracy: 0.7175
Epoch 281/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0213 - accuracy: 1.0000 - val_loss: 0.9194 - val_accuracy: 0.7225
Epoch 282/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.0225 - accuracy: 1.0000 - val_loss: 0.9418 - val_accuracy: 0.7325
Epoch 283/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.0227 - accuracy: 1.0000 - val_loss: 0.9232 - val_accuracy:

Epoch 334/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0153 - accuracy: 1.0000 - val_loss: 0.9767 - val_accuracy: 0.7150
Epoch 335/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0142 - accuracy: 1.0000 - val_loss: 0.9757 - val_accuracy: 0.7125
Epoch 336/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0139 - accuracy: 1.0000 - val_loss: 0.9771 - val_accuracy: 0.7150
Epoch 337/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0144 - accuracy: 1.0000 - val_loss: 1.0023 - val_accuracy: 0.7250
Epoch 338/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0147 - accuracy: 1.0000 - val_loss: 0.9797 - val_accuracy: 0.7225
Epoch 339/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0152 - accuracy: 1.0000 - val_loss: 0.9782 - val_accuracy: 0.7200
Epoch 340/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0128 - accuracy: 1.0000 - val_loss: 0.9778 - val_accuracy:

Epoch 391/1000
63/63 [==============================] - 0s 5ms/step - loss: 0.0100 - accuracy: 1.0000 - val_loss: 1.0162 - val_accuracy: 0.7125
Epoch 392/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.0103 - accuracy: 1.0000 - val_loss: 1.0308 - val_accuracy: 0.7225
Epoch 393/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.0101 - accuracy: 1.0000 - val_loss: 1.0247 - val_accuracy: 0.7175
Epoch 394/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.0101 - accuracy: 1.0000 - val_loss: 1.0229 - val_accuracy: 0.7200
Epoch 395/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0121 - accuracy: 1.0000 - val_loss: 1.0315 - val_accuracy: 0.7200
Epoch 396/1000
63/63 [==============================] - 0s 5ms/step - loss: 0.0103 - accuracy: 1.0000 - val_loss: 1.0222 - val_accuracy: 0.7125
Epoch 397/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.0094 - accuracy: 1.0000 - val_loss: 1.0242 - val_accuracy:

Epoch 448/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.0079 - accuracy: 1.0000 - val_loss: 1.0586 - val_accuracy: 0.7175
Epoch 449/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.0075 - accuracy: 1.0000 - val_loss: 1.0772 - val_accuracy: 0.7225
Epoch 450/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0076 - accuracy: 1.0000 - val_loss: 1.0637 - val_accuracy: 0.7225
Epoch 451/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0075 - accuracy: 1.0000 - val_loss: 1.0633 - val_accuracy: 0.7150
Epoch 452/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0072 - accuracy: 1.0000 - val_loss: 1.0647 - val_accuracy: 0.7150
Epoch 453/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.0075 - accuracy: 1.0000 - val_loss: 1.0627 - val_accuracy: 0.7150
Epoch 454/1000
63/63 [==============================] - 0s 4ms/step - loss: 0.0077 - accuracy: 1.0000 - val_loss: 1.0708 - val_accuracy:

Epoch 505/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0059 - accuracy: 1.0000 - val_loss: 1.1077 - val_accuracy: 0.7200
Epoch 506/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0059 - accuracy: 1.0000 - val_loss: 1.1022 - val_accuracy: 0.7225
Epoch 507/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0064 - accuracy: 1.0000 - val_loss: 1.0968 - val_accuracy: 0.7200
Epoch 508/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0062 - accuracy: 1.0000 - val_loss: 1.0995 - val_accuracy: 0.7225
Epoch 509/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0061 - accuracy: 1.0000 - val_loss: 1.1062 - val_accuracy: 0.7175
Epoch 510/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0061 - accuracy: 1.0000 - val_loss: 1.1038 - val_accuracy: 0.7175
Epoch 511/1000
63/63 [==============================] - 0s 3ms/step - loss: 0.0060 - accuracy: 1.0000 - val_loss: 1.1003 - val_accuracy: